## Tratamiendo de los datos de SONDA

In [1]:

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math

from helper_fn import preprocess_features, preprocess_new_features


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Preprocesamiento de los datos

Primero, cargamos los datos desde un archivo CSV

In [2]:
oportunidades = pd.read_csv("oportunidades2.csv")
to_predict_file= pd.read_csv("Libro14.csv")
to_predict_file.drop(['Unnamed: 10'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,4,6,9,10,14,15,17,19,20,21,22,25,27,28,30,33,34,35,36,43,44,48,49,50,54,55,56,57,61,62,63,70,72,74,78,80,89,90,91,96,97,98,100,109,111,112,119,122,124,126,127,130,133,139,140,141,144,164,171,177,201,236) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:

#to_predict_file

La siguiente función toma el archivo ``oportunidades.csv`` y selecciona los atributos utiles para este analisis

Buscamos los dos sets de filial crm y filial no crm

In [4]:
filial_crm, filial_nocrm = preprocess_features(oportunidades)
to_predict_features = preprocess_new_features(to_predict_file)

n = len(filial_crm.columns)

filial_crm.drop(['cmtx_unidaddenegocioname', 'Duracion_dias', 'Estado_Perdido'], axis=1, inplace=True)
to_predict_features.drop(['cmtx_unidaddenegocioname'], axis=1, inplace=True)


/Users/atescobar/Development/projects/sonda/datosCompletos/helper_fn.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  selected_features['cmtx_fechacierreoportunidad'] = pd.to_datetime(selected_features['cmtx_fechacierreoportunidad'])
/Users/atescobar/Development/projects/sonda/datosCompletos/helper_fn.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  selected_features['createdon'] = pd.to_datetime(selected_features['createdon'])


In [5]:
to_predict_features.dtypes

Opportunityid              float64
totalamount_base           float32
cmtx_commitedtoforecast    float64
relevante                    int64
period_1quarter              int64
period_2quarter              int64
period_3quarter              int64
period_4quarter              int64
period_overayear             int64
dtype: object

## Sets de prueba y de entrenamiento
Dividimos la muestra en prueba y entrenamiento, dejando de entrenamiento los datos del 2018. Primero trabajaremos solo con las filiales que son parte del crm, para las cuales tenemos datos de clasificación sensibles.

In [6]:

thisyear = pd.to_datetime('today').year
year_str = '1/1/'+str(thisyear)
thisyear = pd.to_datetime(year_str)
#ts = thisyear - pd.DateOffset(years=1)
ts = thisyear - pd.DateOffset(years=1)



#to_predict_features = filial_crm.loc[:, filial_crm.columns != 'Estado_Ganado'].loc[filial_crm.index > thisyear]
to_predict_id = to_predict_features['Opportunityid']
to_predict_features.drop(['Opportunityid'],axis=1, inplace=True)
to_train_features = filial_crm.loc[:, filial_crm.columns != 'Estado_Ganado'].loc[filial_crm.index <= thisyear]
to_train_target = filial_crm['Estado_Ganado'].loc[filial_crm.index <= thisyear]
to_train_features.drop(['opportunityid'],axis=1, inplace=True)



train = to_train_features
target = to_train_target


#montototal = filial_crm[['totalamount_base', 'Estado_Ganado']]
#monto = filial_crm[['totalamount_base', 'Estado_Ganado']]
#montototal = montototal.loc[montototal['Estado_Ganado'] == 1]
#montototal = montototal.loc[montototal.index >= thisyear]
#montototal = montototal.loc[montototal.index < thisyear]
#filial_crm.drop(['totalamount_base'],axis=1, inplace=True)
#montototal.drop(['Estado_Ganado'],axis=1, inplace=True)


## Crear el modelo


In [7]:


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(10).repeat().batch(batch_size)
    return dataset




def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    return dataset



In [8]:
feature_columns = []

for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))


In [9]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[2*n+1 ,],
    n_classes=2,
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpzx37bqbp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1f177e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train the model

In [21]:
batch_size = 100
train_steps = 500
 
classifier.train(input_fn = lambda: train_input_fn(train, target, batch_size), steps = train_steps)
#eval_result = classifier.evaluate(input_fn = lambda: eval_input_fn(to_predict_features, None,batch_size))
#print(eval_result)

#accuracy_score = classifier.evaluate(input_fn=lambda:eval_input_fn(features_test,labels=target_test,batch_size=batch_size))['accuracy']
#print('\nTest Accuracy: {0:f}%\n'.format(accuracy_score*100))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpzx37bqbp/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpzx37bqbp/model.ckpt.
INFO:tensorflow:loss = 92064770.0, step = 1001
INFO:tensorflow:global_step/sec: 381.006
INFO:tensorflow:loss = 140281460.0, step = 1101 (0.264 sec)
INFO:tensorflow:global_step/sec: 578.47
INFO:tensorflow:loss = 48307156.0, step = 1201 (0.173 sec)
INFO:tensorflow:global_step/sec: 529.185
INFO:tensorflow:loss = 146635180.0, step = 1301 (0.188 sec)
INFO:tensorflow:global_step/sec: 584.99
INFO:tensorflow:loss = 266021860.0, step = 1401 (0.171 sec)
INFO:tensorflow:Saving checkpoints for 1500 into /var/folders/gw/mpmkzqcs0

# Predicciones

Ahora vemos como se comporta en la predicción de cierre de oportunidad

In [22]:

predictions = classifier.predict(input_fn=lambda:eval_input_fn(to_predict_features,labels=None, batch_size=batch_size))

template = ('\nPrediction is {} with ({:.1f}%) probability')

val = []
val2 = []
val3 = []
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0] 
    if(class_id==0):
        probability = 1-pred_dict['probabilities'][class_id]
    if(class_id==1):
        probability = pred_dict['probabilities'][class_id]
    val2.append(class_id)
    val3.append(probability)
    
    #print(template.format(class_id, 100 * probability))

print("\nPredictions stored in array val")


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpzx37bqbp/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Predictions stored in array val


In [23]:
val1 = to_predict_id.as_matrix()

df = pd.DataFrame({'opt_id': val1, 'class_id': val2, 'prob':val3})
df['opt_id'] = df['opt_id'].astype(int)
df = df.set_index('opt_id')
df.to_csv('results.csv')

df

,class_id,prob
opt_id,,
202219,0,0.478104
212552,1,0.994810
210768,1,0.934218
215400,0,0.005605
215930,0,0.478104
214446,0,0.065968
210592,0,0.000000
219183,0,0.015285
219235,1,0.789920
